In [3]:
import numpy as np
import sympy
emc = float( sympy.S.EulerGamma.n(10) )
from tteVAMP.problem import Problem
from tteVAMP.simulations import *
from tteVAMP.vamp import infere
from tteVAMP.utils import plot_metrics
import pandas as pd
import zarr
import os

In [6]:
np.random.seed(42)
n=800
m=800
p=0.4
la=0.05
# sigma=1
h2=0.5


mu=np.full((n,1), 0) 
X = sim_geno(n,m,p)
column_means = np.nanmean(X, axis=0)
column_stds = np.nanstd(X, axis=0)
print(f"Are standard deviations valid? {not 0 in column_stds}")
X = (X - column_means) / column_stds
X = np.nan_to_num(X)
variances = np.var(X[:, :5], axis=0)
beta = sim_beta(m, la, h2/m/la)
y, alpha = sim_pheno_Weibull(X, beta, mu, h2)
print(f"Generated alpha value is: {alpha}")
print(f"Variance of Xb: {np.var(X@beta)}")
print(f"Variance of b: {np.var(beta)}")

Are standard deviations valid? True
Variance of beta: 0.012814499328007614
Generated alpha value is: 1.567078222875754
Variance of Xb: 0.669833932317008
Variance of b: 0.000757771590922301


In [16]:
if not os.path.exists('gibbs'):
    os.makedirs('gibbs')


In [17]:
indices =  np.arange(0,m)
df = pd.DataFrame({'IID': indices, 'FID': indices, 'y': y.squeeze(-1)})
df.to_csv("./gibbs/800x800_h2_0.6_la_0.5.phen", sep='\t', index=None, header=None)

In [18]:
vector = np.ones(m)
np.savetxt('./gibbs/status.fail', vector, fmt='%d')

In [19]:
zarr.save('./gibbs/X.zarr', X)